# Import libs

In [1]:
# Import some stuff
import os
from dotenv import load_dotenv
from pathlib import Path
from pymongo import MongoClient
import matplotlib.pyplot as plt
import pandas as pd
import ast
import datetime
import seaborn as sns
import json
import numpy as np

import matplotlib as mpl
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

from datetime import datetime
from dateutil.parser import parse
import pandas as pd


from pandas.io.json import json_normalize
from PIL import Image


#need to translate from wgs to bng to do spatial clustering
import pyproj

In [2]:
import folium

In [3]:
#plotly info
plotly.tools.set_credentials_file(username='aclong', api_key='A9dlryDYONXram1rzbki')

In [4]:
# specifies number of rows to show
pd.set_option('display.max_rows', 300) 

# specifies default number format to 4 decimal places
pd.options.display.float_format = '{:40,.4f}'.format 

# specifies that graphs should use ggplot styling
plt.style.use('ggplot') 

%matplotlib inline

In [5]:
# load .env variable
# base_dir = 'D:\Documentos\GitHub\CityBlender'
# dotenv_file = os.path.join(base_dir, '.env')

base_dir = 'C:/Users/Alfie/Documents/MSc Smart Cities/Term 2/Spatial Data Capture Storage and Analysis/Coursework/Analysis Test/'
dotenv_file = os.path.join(base_dir, '.env')


if os.path.isfile(dotenv_file):
    load_dotenv(dotenv_file, verbose=True)
    
db_uri = os.getenv('DB_URI')

In [6]:
# create a new MongoDB Client
db_client = MongoClient(str(db_uri))

# If it's ok it should return a 1.0
db_client.london.command("serverStatus")['ok']

ServerSelectionTimeoutError: none:27017: [Errno 11001] getaddrinfo failed

In [ ]:
# choose a database to connect to (.london)
db_london = db_client.london

# choose a collection
db_london_events = db_london['events']
db_london_artist = db_london['artists']

In [ ]:
# lookup all the documents in a collection
db_london_events.find()

## EOD

In [ ]:
events = pd.DataFrame(list(db_london_events.find({})))

In [ ]:
events.columns

### For instance in the first row there are 3 events

In [ ]:
# Put this in the date fomrat
events['date'] = [(datetime.strptime(events['date'][x], '%Y-%m-%d')) for x in range(len(events['date']))]

# Convert the dates into day name
events['date_name'] = [(events['date'][x].strftime("%A")) for x in range(len(events['date']))]

# Lat & Long
events['lat'] = [(events['location'][x]['lat']) for x in range(len(events['location']))]
events['lng'] = [(events['location'][x]['lng']) for x in range(len(events['location']))]

# Plotly Stuff

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly

plotly.tools.set_credentials_file(username='cohenjota', api_key='I6zLnEN42oNTpZv36icg')

## City Level

In [ ]:
import fiona
from mpl_toolkits.basemap import Basemap   
import pandas as pd
from shapely.geometry import Polygon, Point, MultiPoint, MultiPolygon
from shapely.prepared import prep
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.colors import Normalize
from descartes import PolygonPatch

# DB SCAN

In [ ]:
#clustering
from sklearn.cluster import DBSCAN
from scipy import cluster

In [ ]:
plt.scatter(events['lng'], events['lat'])

In [ ]:
subset = events[events['lat']<52]

In [ ]:
plt.scatter(subset['lng'], subset['lat'])

In [ ]:
subset = subset[subset['lat']>51.2]

In [ ]:
plt.scatter(subset['lng'], subset['lat'])

In [ ]:
def SCANING_DB(eps_var = 0.1 , min_sample_var = 5, db = subset, lat_var = 'lat', lng_var = 'lng'):
    dbscan = DBSCAN(eps=eps_var, min_samples = min_sample_var)
#     print(dbscan) 
    arrey_coords = db[[lat_var, lng_var]].values
    #print(arrey_coords)
    #try clustering out on the whole dataset
    dbscan_result = dbscan.fit(arrey_coords)
    dbscan_result_lbls = dbscan_result.labels_
    return(dbscan_result_lbls)
    
    

In [ ]:
fitted = SCANING_DB(eps_var = 0.001)
subset['cluster'] = fitted
print(len(subset['cluster'].unique()))
print(subset.groupby('cluster').count()['_id'].sort_values(ascending = False))
subset['cluster'].hist()

In [ ]:
fitted = SCANING_DB(eps_var = 0.01)
subset['cluster'] = fitted
print(len(subset['cluster'].unique()))
print(subset.groupby('cluster').count()['_id'].sort_values(ascending = False))
subset['cluster'].hist()

In [ ]:
fitted = SCANING_DB(eps_var = 0.025)
subset['cluster'] = fitted
print(len(subset['cluster'].unique()))
print(subset.groupby('cluster').count()['_id'].sort_values(ascending = False))
subset['cluster'].hist()

In [ ]:
fitted = SCANING_DB(eps_var = 0.06)
subset['cluster'] = fitted
print(len(subset['cluster'].unique()))
print(subset.groupby('cluster').count()['_id'].sort_values(ascending = False))
subset['cluster'].hist()

# Convert to BNG

In [ ]:
#need to translate from wgs to bng to do spatial clustering
import pyproj

In [ ]:
#to do clustering locations need to be in BNG so that distances are in metres
#Define projections using EPSG codes
wgs84=pyproj.Proj("+init=EPSG:4326") # LatLon with WGS84 datum used by songkick
osgb36=pyproj.Proj("+init=EPSG:27700") # UK Ordnance Survey, 1936 datum, for locations in eastings/northings/metres

#create a new pd without NaNs
londonEventsdfNoNan = subset[subset['lat'].notnull()]

#make new columns for the new latlng
londonEventsdfNoNan['BNGnorthing'] = ""
londonEventsdfNoNan['BNGeasting'] = ""

#reset the index - londonEventsdfNoNan now main dataframe
londonEventsdfNoNan = londonEventsdfNoNan.reset_index()

In [ ]:
#run through the dataset and transform those coordinates
origLat = []
origLng = []

for i in range(0,len(londonEventsdfNoNan)):
    # print(i)
    try:
        origLat.append(londonEventsdfNoNan['lat'][i])
        origLng.append(londonEventsdfNoNan['lng'][i])
        
    except:
        origLat.append(np.nan)
        origLng.append(np.nan)

In [ ]:
lat,lon = pyproj.transform(wgs84,osgb36,origLng,origLat)
    
londonEventsdfNoNan['BNGeasting'] = lat
londonEventsdfNoNan['BNGnorthing'] = lon

In [ ]:
#now convert from object to numeric
londonEventsdfNoNan['BNGnorthing'] = pd.to_numeric(londonEventsdfNoNan['BNGnorthing'])
londonEventsdfNoNan['BNGeasting'] = pd.to_numeric(londonEventsdfNoNan['BNGeasting'])

In [ ]:
londonEventsdfNoNan

In [ ]:
plt.scatter(londonEventsdfNoNan['lng'], londonEventsdfNoNan['lat'])

In [ ]:
plt.scatter(londonEventsdfNoNan['BNGeasting'], londonEventsdfNoNan['BNGnorthing'])

In [ ]:
plt.scatter(londonEventsdfNoNan['lat'], londonEventsdfNoNan['BNGnorthing'])

In [ ]:
plt.scatter(londonEventsdfNoNan['lng'], londonEventsdfNoNan['BNGeasting'])

In [ ]:
fitted = SCANING_DB(eps_var = 750, lat_var = 'BNGnorthing', lng_var = 'BNGeasting', db = londonEventsdfNoNan)
londonEventsdfNoNan['cluster'] = fitted
print(len(londonEventsdfNoNan['cluster'].unique()))
print(londonEventsdfNoNan.groupby('cluster').count()['_id'].sort_values(ascending = False))
londonEventsdfNoNan['cluster'].hist()

In [ ]:
# Make a function to subset the genre depending on which dataset
def subset_by_genre(db, lib = 'spotify', words = 'reggae'):
    return db[db[lib].astype(str).str.contains(words, case=False)]

#create a genre compiling function
def genre_df_compiler(genreName, db = londonEventsdfNoNan):
    firstEventsdf = subset_by_genre(db, lib='spotify',words = genreName)
    secondEventsdf = subset_by_genre(db, lib='lastfm',words = genreName)
    allEventsdf = "all"+genreName+"Events" 
    allEventsdf = pd.concat([firstEventsdf, secondEventsdf])
    allEventsdf = allEventsdf.drop_duplicates(subset='_id', keep='last')
    return(allEventsdf)

In [ ]:
jazz =  genre_df_compiler('jazz', db = londonEventsdfNoNan)
jazz.reset_index(inplace = True)

In [ ]:
jazz

# Test jazz

In [ ]:
fitted = SCANING_DB(eps_var = 750, lat_var = 'BNGnorthing', lng_var = 'BNGeasting', db = jazz)
jazz['cluster'] = fitted
print(len(jazz['cluster'].unique()))
print(jazz.groupby('cluster').count()['_id'].sort_values(ascending = False))
jazz['cluster'].hist()

In [ ]:
#create dataframe for storing the clusternos per eps value
clusterNoEps = pd.DataFrame()

In [ ]:
def eps_df_genre(db = jazz, epsMin = 1, epsMax = 2000, epsStep = 10, colName = "ClusterNosjazz"):
    clusterNos = []
    epsValues = list(range(epsMin, epsMax, epsStep))
    epsDF = pd.DataFrame(epsValues)
    for i in epsValues:
        fitted = SCANING_DB(eps_var = i, lat_var = 'BNGnorthing', lng_var = 'BNGeasting', db = db)
        db['cluster'] = fitted
        clusterNos.append(len(db['cluster'].unique()))
    epsDF[colName] = pd.DataFrame(clusterNos)
    return epsDF

In [ ]:
clusterNos

In [ ]:
epsDFjazz = eps_df_genre(db = jazz, colName = "ClusterNosjazz")


In [ ]:
epsDFevents = eps_df_genre(db = londonEventsdfNoNan, colName = "ClusterNosevents")

In [ ]:
epsDFjazz

In [ ]:
folk =  genre_df_compiler('folk', db = londonEventsdfNoNan)
folk.reset_index(inplace = True)
epsDFfolk = eps_df_genre(db = folk, colName = "ClusterNosfolk")

In [ ]:
indie =  genre_df_compiler('indie', db = londonEventsdfNoNan)
indie.reset_index(inplace = True)
epsDFindie = eps_df_genre(db = indie, colName = "ClusterNosindie")

In [ ]:
classical =  genre_df_compiler('classical', db = londonEventsdfNoNan)
classical.reset_index(inplace = True)
epsDFclassical = eps_df_genre(db = classical, colName = "ClusterNosclassical")

In [ ]:
house =  genre_df_compiler('house', db = londonEventsdfNoNan)
house.reset_index(inplace = True)
epsDFhouse = eps_df_genre(db = house, colName = "ClusterNoshouse")

In [ ]:
rock =  genre_df_compiler('rock', db = londonEventsdfNoNan)
rock.reset_index(inplace = True)
epsDFrock = eps_df_genre(db = rock, colName = "ClusterNosrock")

In [ ]:
reggae =  genre_df_compiler('rock', db = londonEventsdfNoNan)
reggae.reset_index(inplace = True)
epsDFreggae = eps_df_genre(db = reggae, colName = "ClusterNosreggae")

In [ ]:
grime =  genre_df_compiler('grime', db = londonEventsdfNoNan)
grime.reset_index(inplace = True)
epsDFgrime = eps_df_genre(db = grime, colName = "ClusterNosgrime")

In [ ]:
epsDFall = pd.concat([epsDFevents, epsDFjazz, epsDFfolk, epsDFclassical, epsDFgrime, epsDFreggae, epsDFrock, epsDFhouse, epsDFindie], axis=1, join_axes=[epsDFevents[0]])

In [ ]:
epsDFall

In [ ]:
epsDFall = epsDFall.loc[:,~epsDFall.columns.duplicated()]

In [ ]:
epsDFall.columns

In [ ]:
trace0 = go.Scatter(x=epsDFall[0], 
                    y = epsDFall['ClusterNosevents'], 
                    name = 'All Events', 
                    line = dict(color = ('rgb(205, 12, 24)'), width = 4))
trace1 = go.Scatter(x=epsDFall[0], 
                    y = epsDFall['ClusterNosjazz'], 
                    name = 'Jazz Events', 
                    line = dict(color = ('rgb(22, 96, 167)'), width = 4))
trace2 = go.Scatter(x=epsDFall[0], 
                    y = epsDFall['ClusterNosfolk'], 
                    name = 'Folk Events', 
                    line = dict(color = ('rgb(100, 12, 24)'), width = 4))
trace3 = go.Scatter(x=epsDFall[0], 
                    y = epsDFall['ClusterNosclassical'], 
                    name = 'Classical Events', 
                    line = dict(color = ('rgb(150, 96, 167)'), width = 4))
trace4 = go.Scatter(x=epsDFall[0], 
                    y = epsDFall['ClusterNosgrime'], 
                    name = 'Grime Events', 
                    line = dict(color = ('rgb(70, 96, 167)'), width = 4))
trace5 = go.Scatter(x=epsDFall[0], 
                    y = epsDFall['ClusterNosreggae'], 
                    name = 'Reggae Events', 
                    line = dict(color = ('rgb(175, 96, 167)'), width = 4))
trace6 = go.Scatter(x=epsDFall[0], 
                    y = epsDFall['ClusterNosrock'], 
                    name = 'Rock Events', 
                    line = dict(color = ('rgb(255, 96, 167)'), width = 4))
trace7 = go.Scatter(x=epsDFall[0], 
                    y = epsDFall['ClusterNoshouse'], 
                    name = 'House Events', 
                    line = dict(color = ('rgb(0, 96, 167)'), width = 4))
trace8 = go.Scatter(x=epsDFall[0], 
                    y = epsDFall['ClusterNosindie'], 
                    name = 'Indie Events', 
                    line = dict(color = ('rgb(70, 96, 100)'), width = 4))

graphData = [
#             trace0, 
             trace1, 
             trace2, 
             trace3, 
             trace4,
             trace5,
             trace6,
             trace7,
             trace8]

# Edit the layout
layout = dict(title = 'No. Clusters at EPS values',
              xaxis = dict(title = 'EPS Value'),
              yaxis = dict(title = 'No. Clusters'),
              )

fig = dict(data=graphData, layout=layout)
py.iplot(fig, filename='styled-line')

In [ ]:
#now get folium map of clusters
#need to create the function to add labels of clusters to the dataframe
#eps 500 minsamples 5
# def SCANING_DB(eps_var = 0.1 , min_sample_var = 5, db = subset, lat_var = 'lat', lng_var = 'lng'):
#     dbscan = DBSCAN(eps=eps_var, min_samples = min_sample_var)
#     print(dbscan) 
#     arrey_coords = db[[lat_var, lng_var]].values
    #print(arrey_coords)
# try clustering out on the whole dataset
#     dbscan_result = dbscan.fit(arrey_coords)
#     dbscan_result_lbls = dbscan_result.labels_
#     return(dbscan_result_lbls)

dbscanJazzLabels = SCANING_DB(eps_var = 500, db = jazz, lat_var = 'BNGnorthing', lng_var = 'BNGeasting')

jazz['dbscan500by5JazzEvents'] = pd.DataFrame(dbscanJazzLabels)

jazz['dbscan500by5JazzEvents'].hist()

jazz.plot.scatter(y='BNGnorthing', x='BNGeasting', c='dbscan500by5JazzEvents')

In [ ]:
#find the different labels inside the db
jazz.dbscan500by5JazzEvents.unique()

In [ ]:
#possible colors
colors = [
    'red',
    'blue',
    'gray',
    'darkred',
    'lightred',
    'orange',
    'beige',
    'green',
    'darkgreen',
    'lightgreen',
    'darkblue',
    'lightblue',
    'purple',
    'darkpurple',
    'pink',
    'cadetblue',
    'lightgray',
    'black']


In [ ]:
#new jazz list
#trying out a map of this clustering
#create a map with our data in center
latList = jazz['lat'].tolist()
lngList = jazz['lng'].tolist()
clusterLabelList = jazz['dbscan500by5JazzEvents'].tolist()
medLat = jazz['lat'].median()
medLng = jazz['lng'].median()

this_map = folium.Map([medLat, medLng], zoom_start=12, tiles="Cartodb Positron", control_scale=True)




for lat, lng, label in zip(latList, lngList, clusterLabelList):
    if label == -1:
        this_map.add_child(folium.CircleMarker([lat, lng], 
#                                            popup=label, 
                                           radius = 5, color='gray', fill_color='orange', fill_opacity=0.7))
    elif label == 0:
        this_map.add_child(folium.CircleMarker([lat, lng], 
#                                            popup=label, 
                                           radius = 5, color='blue', fill_color='orange', fill_opacity=0.7))
    elif label == 1:
        this_map.add_child(folium.CircleMarker([lat, lng], 
#                                            popup=label, 
                                           radius = 5, color='lightred', fill_color='orange', fill_opacity=0.7))
    elif label == 2:
        this_map.add_child(folium.CircleMarker([lat, lng], 
#                                            popup=label, 
                                           radius = 5, color='yellow', fill_color='orange', fill_opacity=0.7))
    elif label == 3:
        this_map.add_child(folium.CircleMarker([lat, lng], 
#                                            popup=label, 
                                           radius = 5, color='lightgreen', fill_color='orange', fill_opacity=0.7))
    elif label == 4:
        this_map.add_child(folium.CircleMarker([lat, lng], 
#                                            popup=label, 
                                           radius = 5, color='orange', fill_color='orange', fill_opacity=0.7))
    elif label == 5:
        this_map.add_child(folium.CircleMarker([lat, lng], 
#                                            popup=label, 
                                           radius = 5, color='purple', fill_color='orange', fill_opacity=0.7))
    elif label == 6:
        this_map.add_child(folium.CircleMarker([lat, lng], 
#                                            popup=label, 
                                           radius = 5, color='pink', fill_color='orange', fill_opacity=0.7))
    elif label == 7:
        this_map.add_child(folium.CircleMarker([lat, lng], 
#                                            popup=label, 
                                           radius = 5, color='lightblue', fill_color='orange', fill_opacity=0.7))
    elif label == 8:
        this_map.add_child(folium.CircleMarker([lat, lng], 
#                                            popup=label, 
                                           radius = 5, color='black', fill_color='orange', fill_opacity=0.7))
    elif label == 9:
        this_map.add_child(folium.CircleMarker([lat, lng], 
#                                            popup=label, 
                                           radius = 5, color='darkred', fill_color='orange', fill_opacity=0.7))
    elif label == 10:
        this_map.add_child(folium.CircleMarker([lat, lng], 
#                                            popup=label, 
                                           radius = 5, color='beige', fill_color='orange', fill_opacity=0.7))
    elif label == 11:
        this_map.add_child(folium.CircleMarker([lat, lng], 
#                                            popup=label, 
                                           radius = 5, color='cadetblue', fill_color='orange', fill_opacity=0.7))
    elif label == 12:
        this_map.add_child(folium.CircleMarker([lat, lng], 
#                                            popup=label, 
                                           radius = 5, color='darkgreen', fill_color='orange', fill_opacity=0.7))
    
    else:
        pass


In [ ]:
this_map